In [ ]:
!pip install transformers datasets evaluate seqeval
!pip install transformers datasets evaluate accelerate

In [ ]:
#Load Dataset
from datasets import load_dataset

dataset = load_dataset("thomasavare/waste-classification-v2")
dataset = dataset.remove_columns(['Unnamed: 0'])
dataset = dataset.rename_column("Phrase", "text")
dataset = dataset.rename_column("Class", "label")

id2label = res = {dataset['train'][i]['Class_index'] : dataset['train'][i]['label'] for i in range(len(dataset['train']['Class_index']))}
label2id= {dataset['train'][i]['label'] : dataset['train'][i]['Class_index'] for i in range(len(dataset['train']['Class_index']))}

dataset = dataset.remove_columns(['label'])
dataset = dataset.rename_column("Class_index", "label")





In [ ]:
#Tokenization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_db = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/21528 [00:00<?, ? examples/s]

Map:   0%|          | 0/5382 [00:00<?, ? examples/s]

In [ ]:

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_db["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=50, id2label=id2label, label2id=label2id
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_db["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_db["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [ ]:


model.compile(optimizer=optimizer, metrics='accuracy')  # No loss argument!





['text', 'label', 'input_ids', 'attention_mask']